<a href="https://colab.research.google.com/github/kevingray92/597_Project4_GroupB/blob/dailyobscleanup/Project4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import numpy as np
import pandas as pd
import os
# import tarfile

### Adding hourly KCMI precip to the daily data

In [ ]:
#Download the data
!wget https://raw.githubusercontent.com/swnesbitt/ATMS-597-SP-2020/master/ATMS-597-SP-2020-Project-4/KCMI_hourly.csv

In [ ]:
#Used to unzip the .tar.gz files
!gunzip drive/My\ Drive/Colab\ Notebooks/ATMS-597/Module04/Project04/sfc.tar.gz

In [ ]:
#Use this to untar the .tar files
#Looks like it will send all of these to a new remote directory called 'bufkit'
#Might want to specify a destination folder in the future
!tar -xvf drive/My\ Drive/Colab\ Notebooks/ATMS-597/Module04/Project04/sfc.tar

In [3]:
# dailyobs = pd.read_csv('drive/My Drive/KCMI_dailyNOFOOTER.csv', header=8)  # Read the daily obs (removed the footer in the original file)
dailyobs = pd.read_csv('./raw_data/KCMI_dailyNOFOOTER.csv', header=8)  # (David) Read the daily obs (removed the footer in the original file)
dailyobs.drop(['Total Precip (in)'],axis=1)  # Drop the daily precip column that we are going to replace
dailyobs

,Date,Max Hourly Temp (F),Min Hourly Temp (F),Max Wind Speed (mph),Total Precip (in)
0,1/1/2010,12,2,15,M
1,1/2/2010,6,-5,15,0
2,1/3/2010,10,-6,11,0
3,1/4/2010,11,-2,20,0
4,1/5/2010,10,-1,16,M
...,...,...,...,...,...
3647,12/27/2019,43,32,15,0
3648,12/28/2019,56,32,23,0.23
3649,12/29/2019,59,41,23,0.22
3650,12/30/2019,38,31,26,0.01


In [5]:
# hourlyobs = pd.read_csv('drive/My Drive/KCMI_hourly.csv')  # Read the hourly file
hourlyobs = pd.read_csv('./raw_data//KCMI_hourly.csv')  # (David) Read the hourly file
hourlyprecip = hourlyobs.drop(['year','month','day','hour','tmpc','dwpc','mslp','wdir','wspd','skct','pr6h','doy','woy'],axis=1)  # Only keep precip column
hourlyprecip.set_index('Timestamp')  # Make the index be the timestamp
hourlyprecip.index = pd.to_datetime(hourlyprecip['Timestamp'])  # Make the index be a datetime
dailyprecip = hourlyprecip.resample('d').sum()  # Resample to daily
dailyprecip.columns = ['Daily Precip (mm)']  # Rename the column

In [8]:
dailyprecip.head()

,Daily Precip (mm)
Timestamp,
2005-12-31,0.0
2006-01-01,0.0
2006-01-02,2.3
2006-01-03,0.2
2006-01-04,0.0


In [9]:
dailyprecipshort = dailyprecip[(dailyprecip.index>'2009-12-31') & (dailyprecip.index<'2020-01-01')]  # Only get 2010 to 2019
dailyobs['Daily Precip (mm)'] = dailyprecipshort['Daily Precip (mm)'].values  # Put the new calculated daily precip in the dailyobs dataframe
dailyobs  # This should be the final dataframe used as the predictands

,Date,Max Hourly Temp (F),Min Hourly Temp (F),Max Wind Speed (mph),Total Precip (in),Daily Precip (mm)
0,1/1/2010,12,2,15,M,0.0
1,1/2/2010,6,-5,15,0,0.0
2,1/3/2010,10,-6,11,0,0.0
3,1/4/2010,11,-2,20,0,0.0
4,1/5/2010,10,-1,16,M,-0.1
...,...,...,...,...,...,...
3647,12/27/2019,43,32,15,0,0.0
3648,12/28/2019,56,32,23,0.23,2.7
3649,12/29/2019,59,41,23,0.22,4.0
3650,12/30/2019,38,31,26,0.01,0.4


In [23]:
dailyobs = dailyobs.drop(columns = ['Total Precip (in)']).replace('M', np.nan)

In [25]:
dailyobs = dailyobs.interpolate()

In [34]:
# Convert to deg C, m/s, and set negative precip values to zero
dailyobs['Max Hourly Temp (F)'] = (dailyobs['Max Hourly Temp (F)'].astype(float) - 32.) * 5./9.
dailyobs['Min Hourly Temp (F)'] = (dailyobs['Min Hourly Temp (F)'].astype(float) - 32.) * 5./9.
dailyobs['Max Wind Speed (mph)'] = dailyobs['Max Wind Speed (mph)'].astype(float) * 0.44704
dailyobs['Daily Precip (mm)'] = dailyobs['Daily Precip (mm)'].astype(float).apply(lambda x: 0.0 if x < 0.0 else x)

In [48]:
# Rename columns
dailyobs.rename(columns = {'Max Hourly Temp (F)' : 'Max Hourly Temp (C)', 'Min Hourly Temp (F)' : 'Min Hourly Temp (C)', 'Max Wind Speed (mph)' : 'Max Wind Speed (m/s)'}, inplace = True)

In [53]:
dailyobs

,Date,Max Hourly Temp (C),Min Hourly Temp (C),Max Wind Speed (m/s),Daily Precip (mm)
0,1/1/2010,-11.111111,-16.666667,6.70560,0.0
1,1/2/2010,-14.444444,-20.555556,6.70560,0.0
2,1/3/2010,-12.222222,-21.111111,4.91744,0.0
3,1/4/2010,-11.666667,-18.888889,8.94080,0.0
4,1/5/2010,-12.222222,-18.333333,7.15264,0.0
...,...,...,...,...,...
3647,12/27/2019,6.111111,0.000000,6.70560,0.0
3648,12/28/2019,13.333333,0.000000,10.28192,2.7
3649,12/29/2019,15.000000,5.000000,10.28192,4.0
3650,12/30/2019,3.333333,-0.555556,11.62304,0.4


In [50]:
# Save to csv
dailyobs.to_csv('./raw_data/KCMI_daily_tidy.csv', index = False)

### Parsing the hourly data

In [87]:
# Hourly data
# hourlyobs = pd.read_csv('drive/My Drive/KCMI_hourly.csv')  # Read the hourly file
hourlyobs = pd.read_csv('./raw_data//KCMI_hourly.csv')  # (David) Read the hourly file

In [88]:
hourlyobs = hourlyobs.query('hour >= 18 or hour <= 5')

In [89]:
hourlyobs['Timestamp'] = pd.to_datetime(hourlyobs['Timestamp'])
hourlyobs.set_index('Timestamp', inplace = True)

In [90]:
hourlyobs = hourlyobs.resample('D').mean()

In [91]:
hourlyobs.index = hourlyobs.index + pd.DateOffset(days = 1)

In [92]:
hourlyobs = hourlyobs[(hourlyobs.index > '2009-12-31') & (hourlyobs.index < '2020-01-01')]

In [93]:
hourlyobs = hourlyobs.drop(columns = ['year', 'month', 'day', 'hour', 'pr6h', 'doy', 'woy'])

In [95]:
hourlyobs.to_csv('./raw_data/KCMI_hourly_tidy.csv')

In [94]:
hourlyobs

,tmpc,dwpc,mslp,wdir,wspd,skct,pr1h
Timestamp,,,,,,,
2010-01-01,-7.590909,-11.154545,1022.190909,288.181818,7.036364,2.090909,0.000
2010-01-02,-13.008333,-16.616667,1029.875000,290.833333,6.066667,3.583333,0.000
2010-01-03,-17.091667,-20.558333,1032.716667,306.666667,4.108333,0.000000,0.000
2010-01-04,-14.550000,-18.458333,1031.466667,304.166667,3.933333,0.166667,0.000
2010-01-05,-12.908333,-16.408333,1027.250000,288.333333,7.341667,3.916667,0.000
...,...,...,...,...,...,...,...
2019-12-27,12.225000,10.358333,1017.009091,224.166667,4.366667,1.750000,0.000
2019-12-28,2.166667,0.383333,1025.200000,127.500000,2.066667,0.750000,0.000
2019-12-29,11.258333,10.658333,1013.350000,119.166667,7.016667,9.000000,0.225
